In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export verification

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from tqdm import tqdm
from drt.environment import DataIntakeEnv
import drt.data_model as dm
import drt.utils as utils
from drt.utils import Data_Groups_Type


def check_datagroup(env:DataIntakeEnv,
                    data_group_type: Data_Groups_Type,
                    light:bool = True) -> None:
    """
    A function to check data folders and determine if the data folder registration
    has been properly filled out. It checks for the following things:
        - Have all the data groups in the folder been registered?
        - Were any data groups modified since the last check?
        - Have all the data groups in the folder had their descriptions and origin updated?
        - Warning if a registered delivery is missing

    Parameters
    ----------
    env : DataIntakeEnv

    data_group_type : Data_Groups_Type
        Which type of Data Folder are we checking
        
    light : bool, optional
        Skip file hashing in the comparison

    Example
    -------
    TODO
    [>>> example_usage_of_module in pydoctest]
    """
    if data_group_type == dm.Delivery:
        data_folder = env.delivery_folder
        err_str = 'DLV'
    elif data_group_type == dm.Raw_Data:
        data_folder = env.raw_data_folder
        err_str = 'RAW'
    elif data_group_type == dm.Dataset:
        data_folder = env.dataset_folder
        err_str = 'DAT'
    else:
        raise TypeError

    errors = []

    # Collect the deliveries in the delivery folder
    if data_group_type == dm.Delivery:
        data_groups = [
            fil
            for fil in data_folder.iterdir()
            if fil.is_dir() and not fil.name.startswith(".")
        ]
    else:
        data_groups = [
            fil
            for fil in data_folder.iterdir()
            if fil.is_dir() and not fil.name.startswith(".") and not fil.name.lower().startswith('in_progresss')
        ]
    
    # Loop through all the directories in the deliveries folder to process
    for data_group in tqdm(data_groups, unit="data group", leave=False):

        # Get file sizes, last modified dates, and names to count,
        # sum size, and if light is false, hash the file data provided
        data = None
        try:
            data = utils.process_data_group(data_group, data_group_type, light)
        except FileNotFoundError as e:
            print(f"[{err_str}] Registered data missing files or empty folder at {data_group}")

        # Get the delivery record
        record = env.session.query(data_group_type).filter_by(name=data_group.name).first()

        # Check to see if this delivery has data associated with it. It should have at least 1
        if not record:
            errors.append(f"[{err_str}] No Data registered for: {data_group.name}")
        elif data:
            if data['group_last_modified'] > record.last_update:
                errors.append(f"[{err_str}] Data in {data_group.name} changed since registration")
            ## compare the record to the calculated data
            if not data['size'] == record.size:
                errors.append(f"[{err_str}] Size in {data_group.name} changed since registration") 
            if not data['num_files'] == record.num_files:
                errors.append(f"[{err_str}] Number of files in {data_group.name} changed since registration")     
            if not light:
                if not data['group_hash'] == record.group_hash:
                    errors.append(f"[{err_str}] Hash of {data_group.name} changed since registration") 
            if record.description is None:
                errors.append(f"[{err_str}] {data_group.name} missing description") 

            if data_group_type == dm.Delivery and record.delivery_source is None:
                errors.append(f"[{err_str}] {data_group.name} missing source") 
            if data_group_type == dm.Delivery and record.date_received is None:
                errors.append(f"[{err_str}] {data_group.name} missing delivery date") 
            if data_group_type == dm.Delivery and len(record.extracted_data) == 0:
                errors.append(f"[{err_str}] {data_group.name} no data extracted") 

            if data_group_type == dm.Raw_Data and record.source is None:
                errors.append(f"[{err_str}] {data_group.name} missing source")
            if data_group_type == dm.Raw_Data and record.statistics_report is None:
                errors.append(f"[{err_str}] {data_group.name} missing statistics report")


            if data_group_type == dm.Dataset and record.dataset_report is None:
                errors.append(f"[{err_str}] {data_group.name} missing dataset data sheet")
            
    # retrieve any deliveries that are registered but have no data
    registered = env.session.query(data_group_type).filter(data_group_type.name.notin_([f.name for f in data_groups])).all()
    for item in registered:
        errors.append(f"[{err_str}] {item.name} registered but no data") 

    ## TODO WRITE REPORT OUTPUT HERE
    print("\n".join(errors))

    return None
